In [1]:
import os
import pandas as pd
from dataclasses import  asdict
from datetime import datetime, timedelta
from dscreator.storage import get_storage_handler
from dscreator.sources.ferrybox.extractor import TrajectoryExtractor, NamedTrajectory, NamedArray
from sqlalchemy import create_engine
from dscreator.config import SETTINGS
from dscreator.datasets.trajectories.ferrybox import NorsoopFantasy
from dscreator.sources.ferrybox.uuid_variable_code_mapper import MAPPER
import numpy as np

import xarray as xr

In [2]:
def apply_manual_qc(traj_raw, boat, year) -> NamedTrajectory:
    """Helper to read and apply manually checked data to a named trajectory
    """
    bad_ox = pd.read_csv(f'Norsoop-manual-qc-files/{boat}{year}_bad_ox_sat_dates.txt', header=None)
    bad_T = pd.read_csv(f'Norsoop-manual-qc-files/{boat}{year}_bad_inletT_dates.txt', header=None)
    bad_T = np.array([dt.to_pydatetime() for dt in pd.to_datetime(bad_T[bad_T.columns[0]]).to_list()])
    bad_ox = np.array([dt.to_pydatetime() for dt in pd.to_datetime(bad_ox[bad_ox.columns[0]]).to_list()])
    i_bad_T = np.intersect1d(np.array(traj_raw.datetime_list), bad_T, return_indices=True)[1]
    i_bad_ox = np.intersect1d(np.array(traj_raw.datetime_list), bad_ox, return_indices=True)[1]
    print(f"Found {len(i_bad_T)} timestamp with no flow, and {len(i_bad_ox)} with bad oxygen")
    print(f"{len(np.intersect1d(i_bad_T, i_bad_ox))} timestamps overlap")
    print(f"About to set values for oxygen to None for bad oxygen timestamps")
    traj = NamedTrajectory(
        array_list=[NamedArray(nta.variable_name, [None if i in i_bad_ox else val for i,val in enumerate(nta.values)])
                    if nta.variable_name=="Oxygen" else nta for nta in traj_raw.array_list],
        datetime_list=traj_raw.datetime_list,
        locations=traj_raw.locations)

    print(f"About to remove bad flow data. Before removal size of data based on location is "
          f"{len(traj.locations)}")
    traj = NamedTrajectory(
        array_list=[NamedArray(nta.variable_name, list(np.delete(nta.values, i_bad_T)))
                    for nta in traj.array_list],
        datetime_list=list(np.delete(traj.datetime_list, i_bad_T)),
        locations=list(np.delete(traj.locations, i_bad_T)))
    print(f"After removal size of data based on location is {len(traj.locations)}")

    return traj

In [3]:
boat = "FA"
measurement_parameters = ["Temperature", "Salinity", "Oxygen"]
uuid="14bb8759-81d8-4a1a-948a-14219d374fab"
stationname="color_fantasy"
datasetname=stationname
projectname="NorSoop"

tb = NorsoopFantasy(
    uuid=uuid,
    dataset_name=datasetname,
    station_name=stationname,
    project_name=projectname,
    is_acdd=False
)
years = [2017, 2018, 2019, 2020, 2021, 2022]

In [4]:
engine = create_engine(SETTINGS.database_url)
variable_uuid_map = MAPPER[f"{boat}_19"]
traj_extractor = TrajectoryExtractor(engine, measurement_parameters, variable_uuid_map, [1])
nc_paths = []
for year in years:
    print(f"Extracting year {year}")
    if year == 2020:
        variable_uuid_map = MAPPER[f"{boat}_20"]
        traj_extractor = TrajectoryExtractor(engine, measurement_parameters, variable_uuid_map, [1])

    start_time = datetime(year, 1, 1, 0, 0, 0)
    traj_raw = traj_extractor.fetch_slice(start_time=start_time, end_time=start_time + timedelta(days=365))
    ds = tb.create(apply_manual_qc(traj_raw, boat, year))

    # Set missing value flags
    ds.temperature_qc[ds.temperature.isnull()] = 9
    ds.salinity_qc[ds.salinity.isnull()] = 9
    ds.oxygen_qc[ds.oxygen.isnull()] = 9
    # Store each year on disk after extraction
    SETTINGS.storage_path = os.path.join(os.getcwd(), "..", "catalog")
    sh = get_storage_handler(
        project_name=datasetname,
        dataset_name=stationname,
        unlimited_dims=["time"],
        filename_prefix= f"FA_{year}"
    )
    fname = sh.save_dataset(ds)
    print(f"Dumped {fname.split('/')[-1]}")
    nc_paths.append(fname)

Extracting year 2017


KeyboardInterrupt: 

# Merge each year of data

In [16]:
ds.temperature_qc[ds.temperature ==  5.694] = 9

In [17]:
ds

<xarray.Dataset>
Dimensions:          (time: 600)
Coordinates:
  * time             (time) datetime64[ns] 2022-01-01T13:43:59 ... 2022-01-01...
    longitude        (time) float64 10.25 10.26 10.28 ... 11.63 11.63 11.62
    latitude         (time) float64 54.45 54.45 54.47 ... 57.23 57.23 57.24
Data variables:
    temperature      (time) float64 5.721 5.694 5.633 ... 4.719 4.693 4.635
    salinity         (time) float64 15.36 15.31 15.26 ... 19.65 19.71 19.75
    oxygen           (time) float64 87.08 87.62 88.0 88.13 ... 98.48 98.16 98.21
    temperature_qc   (time) int64 1 9 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    salinity_qc      (time) int64 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    oxygen_qc        (time) int64 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    trajectory_name  <U13 'color_fantasy'
Attributes:
    featureType:  trajectory
    id:           14bb8759-81d8-4a1a-948a-14219d374fab

In [ ]:

ds = xr.merge([xr.open_dataset(p) for p in nc_paths])

refresh attributes based on full dataset

In [ ]:
ds.attrs = asdict(tb.dataset_attributes(ds))
ds

In [ ]:
SETTINGS.storage_path = "gs://nivatest-1-senda"
sh = get_storage_handler(
        project_name=projectname,
        dataset_name=datasetname,
        unlimited_dims=["time"],
        filename_prefix= f"merged"
)
sh.save_dataset(ds)

# Store local csv version

In [ ]:
ds.to_dataframe().to_csv("merged_acdd_color_fantasy.csv")